In [14]:
prompt_template = """You need to generate python code for a synthetic procedural dataset. The dataset is similar to OpenAI's GSM8K which contains grade-school level math questions in natural language.

Here is a the SOURCE item from the dataset which you should translate into a python generator:

```json
{0}
```

As you can see we have already `question_annotated` and `answer_annotated`, but they are not really native python functions. The variable names have these `$` prefixes etc. Could you generate python code which would generate synthetic questions and answers, i.e. with numerical variables from some ranges which would make sense for the given case? Beside the question and answer I also need some metadata e.g. in a dict about the variables used.

I would like to use the generator function later to generate many different variants of of questions and answers based on the same template.
To control the difficulty I want to provide a floating point `difficulty` factor which could be used to scale the numeric ranges .. but please ensure the values integers (e.g cast back to int). If there are variables for which no values are provided like male_names, objects etc. please generate a list of values to sample from that fits  in this context.

1. To make it modular and testable let's split the generator into one function called `generate_from_variables()` which gets the input variables and generates the question and answer texts. It should calculate the answer value from the inputs and the main randomized generator `generate_example()` (see below). 

2. The generator function should have a signature like`def generate_example(rng: Random, difficulty: float = 1.0) -> dict`.

The output dict should contain:
{{
  'question': '<the generated question>',
  'answer': '<the_final_answer>',  # here only the final answer, e.g. the number
  'metadata': {{
    'difficulty': difficulty,
    'answer_value:': <numeric_answer_value>,
    'answer_cot': '<full_long_form_answer>' # chain of thought, similar to 'answer' in the SOURCE
    'variables': {{
        ...  # the variable used
    }}
  }}
}}

3. Write a simple `original_example()` function which calls `generate_from_variables()` and passes the original input values from SOURCE use in the json example above (in order to compare the output).

Your task:

- Generate reasonable random values for all the variables
- Ensure mathematical consistency (total distance is divisible by distance per interval)
- Create natural language question and answer texts
- Include metadata about the variables and solution


I'll provide an example:

INPUT: Original entry from dataset:
```json
{{
  "question": "A fog bank rolls in from the ocean to cover a city. It takes 10 minutes to cover every 3 miles of the city. If the city is 42 miles across from the oceanfront to the opposite inland edge, how many minutes will it take for the fog bank to cover the whole city?",
  "answer": "The city will be covered in 42 / 3 = <<42/3=14>>14 intervals of 10 minutes.\nThus, it will take 14 * 10 = <<14*10=140>>140 minutes for the fog to cover the whole city.\n#### 140",
  "id_orig": 103,
  "id_shuffled": 1,
  "question_annotated": "A fog bank rolls in from the ocean to cover a city. It takes {{t,10}} minutes to cover every {{d,3}} miles of the city. If the city is {{y,42}} miles across from the oceanfront to the opposite inland edge, how many minutes will it take for the fog bank to cover the whole city?\n\n#init:\n- $t = range(2, 500)\n- $d = range(2, 100)\n- $y=range(2, 100)\n\n#conditions:\n- is_int(y/d)\n\n#answer: y//d*t",
  "answer_annotated": "The city will be covered in {{y}}/ {{d}} = <<{{y}}/{{d}}={{y//d}}>>{{y//d}} intervals of {{t}} minutes.\nThus, it will take {{y//d}} * {{t}} = <<{{y//d}}*{{t}}={{y//d*t}}>>{{y//d*t}} minutes for the fog to cover the whole city.\n#### {{y//d*t}}"
}}
```

OUTPUT: Output in the form which should be generated
```python
from random import Random
from typing import Dict, Any

def generate_from_variables(time_per_interval: int, distance_per_interval: int, total_distance: int) -> Dict[str, Any]:
    intervals = total_distance // distance_per_interval
    total_time = intervals * time_per_interval
    
    question = f"A fog bank rolls in from the ocean to cover a city. It takes {{time_per_interval}} minutes to cover every {{distance_per_interval}} miles of the city. If the city is {{total_distance}} miles across from the oceanfront to the opposite inland edge, how many minutes will it take for the fog bank to cover the whole city?"
    
    answer_cot = f"The city will be covered in {{total_distance}} / {{distance_per_interval}} = {{intervals}} intervals of {{time_per_interval}} minutes.\nThus, it will take {{intervals}} * {{time_per_interval}} = {{total_time}} minutes for the fog to cover the whole city.\n#### {{total_time}}"
    
    return {{
        'question': question,
        'answer': f'{{total_time}}',
        'answer_cot': answer,
        'answer_value': total_time,
        'variables': {{
            'time_per_interval': time_per_interval,
            'distance_per_interval': distance_per_interval, 
            'total_distance': total_distance,
            'intervals': intervals
        }}
    }}

def generate_example(rng: Random, difficulty: float = 1.0) -> Dict[str, Any]:
    # Generate random values scaled by difficulty
    distance_per_interval = int(rng.randint(2, int(10 * difficulty)))
    time_per_interval = int(rng.randint(5, int(30 * difficulty)))
    
    # Ensure total distance is divisible by distance_per_interval
    num_intervals = rng.randint(2, int(20 * difficulty))
    total_distance = distance_per_interval * num_intervals
    
    result = generate_from_variables(time_per_interval, distance_per_interval, total_distance)
    
    return {{
        'question': result['question'],
        'answer': result['answer'],
        'metadata': {{
            'answer_cot': result['answer_cot'],
            'difficulty': difficulty,
            'variables': result['variables']
        }}
    }}

def original_example() -> Dict[str, Any]:
   return generate_from_variables(10, 3, 42)
```

Just generate the three python functions for the SOURCE dataset item - no additional explanation.
"""

In [19]:
# create open-router client, place your OPENROUTER_API_KEY in .env file
%load_ext dotenv
%dotenv
import os
import re
from pathlib import Path
from typing import Any, Iterable, Optional
from openai import OpenAI
from openai.types.chat import ChatCompletion, ChatCompletionMessageParam
import time

def llm_generate(
    client: OpenAI,
    messages: Iterable[ChatCompletionMessageParam],
    sampling_params: dict[str, Any],
) -> ChatCompletion:
    max_retry = 3
    for trial in range(max_retry):
        try:
            return client.chat.completions.create(
                messages=messages,
                **sampling_params,
            )
        except Exception as e:
            print("failure response:", e)
            time.sleep(trial * trial)  # quadratic backoff
            if trial == max_retry - 1:
                raise

open_router_client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENROUTER_API_KEY"),
    timeout=90.0,
)

sampling_params = {
    "model": "anthropic/claude-3.5-sonnet",
    "max_tokens": 4096,
}

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [21]:
def generate_simple_request(user_prompt: str, developer_prompt: Optional[str] = None) -> list[dict]:
    prompt = []
    if developer_prompt is not None:
        prompt.append( { "role": "system", "content": developer_prompt } )
    
    prompt.append( { "role": "user", "content": user_prompt })
    return prompt
    

def produce_generator(json_path: Path):
    json = json_path.read_text()
    user_request = prompt_template.format(json)
    
    input_messages = generate_simple_request(user_prompt=user_request)
    output =  llm_generate(open_router_client, input_messages, sampling_params)

    response = output.choices[0].message.content

    return response
    

# clone the gsm-symbolic from apple somewhere and set the path here, `git clone https://github.com/apple/ml-gsm-symbolic.git``
path_to_gsmsym = Path("../../../ml-gsm-symbolic/templates/symbolic/")
print("Reading templates from path: ", path_to_gsmsym.absolute())

template_files = list(path_to_gsmsym.glob("*.json"))

# for testing just do it for the first entry
response_text = produce_generator(template_files[0])

# extract python source section
result_match = re.search(r"^```.*\n((.*\n)+)```", response_text, flags=re.MULTILINE)

pytho_source = result_match.group(1)
pytho_source

/home/koepf/code/open-thought/reasoning-gym/notebooks/../../../ml-gsm-symbolic/templates/symbolic


'from random import Random\nfrom typing import Dict, Any\n\ndef generate_from_variables(item: str, n1: int, p: int, c1: str, c2: str, c3: str) -> Dict[str, Any]:\n    more_cards = int(p/100 * n1)\n    n2 = n1 + more_cards\n    n3 = n1 + n2\n    total = n3 + n3\n\n    question = f"In a set of {item}\'s cards, there are {n1} {c1} cards, and {p}% more {c2} cards. {c3} cards are as many as the sum of {c1} and {c2} cards. How many cards of all mentioned colors are there?"\n\n    answer_cot = f"There are {p}/100 * {n1} = {more_cards} more {c2} cards than {c1} cards.\\n" \\\n                 f"Which means there are {n1} + {more_cards} = {n2} {c2} cards.\\n" \\\n                 f"{c3} cards make up to {n1} + {n2} = {n3} cards.\\n" \\\n                 f"So in total, there are {n3} + {n3} = {total} cards of different colors.\\n" \\\n                 f"#### {total}"\n\n    return {\n        \'question\': question,\n        \'answer\': str(total),\n        \'answer_cot\': answer_cot,\n        \'